#### For Colab

In [ ]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
drive_dir = '/content/drive/My Drive/MAGMA: Summarization/'

#### Install Libraries

In [ ]:
requirements_dir = (drive_dir + 'transformers/examples/seq2seq/').replace(' ', '\ ')
requirements_file = requirements_dir + 'requirements.txt'
!cd $requirements_dir; pip install -r $requirements_file

!pip install transformers==4.1.1
!pip install -U pyarrow
!pip install -U wandb

     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 71kB 2.8MB/s 
     |████████████████████████████████| 7.9MB 11.5MB/s 
     |████████████████████████████████| 7.4MB 31.7MB/s 
     |████████████████████████████████| 327kB 58.5MB/s 
     |████████████████████████████████| 163kB 60.0MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 1.1MB 56.2MB/s 
     |████████████████████████████████| 163kB 58.8MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 112kB 58.7MB/s 
     |████████████████████████████████| 4.5MB 57.6MB/s 
     |████████████████████████████████| 245kB 56.0MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
     |████████████████████████████████| 122kB 54.2MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=0a55e5450220af66b4447787575b5cb0fa274d4d211b1fd29c324baca538c657
  Stored in di

## **Fine-tuning**

In [ ]:
finetune_script = '"'+drive_dir+'transformers/examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval.py"'

### **Config**

In [ ]:
import sys
sys.path.insert(0, drive_dir)
import config

import torch
torch.manual_seed = config.SEED

import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### XSUM

In [ ]:
data_dir = '"/content/drive/My Drive/MAGMA: Summarization/datasets/xsum"'

output_dir = '"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/pegasus_xsum_train"'
log_dir = output_dir + '/logs'

##### Fine tune

In [ ]:
#--warmup_steps 5
#--gradient_accumulation_steps 128
#--do_predict --n_test 20
#--dataloader_drop_last

In [ ]:
!python3 $finetune_script \
--model_name_or_path google/pegasus-large \
--tokenizer_name google/pegasus-large \
--data_dir $data_dir \
--learning_rate 3e-4 --label_smoothing 0.1 \
--sortish_sampler --freeze_embeds --adafactor \
--task summarization \
--max_source_length 1024 \
--max_target_length 60 \
--do_train \
--n_train 80 --num_train_epochs 2 \
--logging_steps 10 --save_steps 40 \
--per_device_train_batch_size 2 --per_device_eval_batch_size 2 \
--do_eval --n_val 20 --evaluation_strategy epoch --val_max_target_length 60 --eval_beams 2 \
--metric_for_best_model rougeL --greater_is_better True \
--predict_with_generate \
--test_max_target_length 100 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $SEED

2020-12-11 10:33:26.661114: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 2.87k/2.87k [00:00<00:00, 1.92MB/s]
Downloading: 100% 1.10k/1.10k [00:00<00:00, 799kB/s]
Downloading: 100% 1.91M/1.91M [00:00<00:00, 21.4MB/s]
Downloading: 100% 2.28G/2.28G [03:35<00:00, 10.6MB/s]
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory /content/drive/My Drive/MAGMA: Summarization/fine-tuning/pegasus_xsum_train exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 2
wandb: You chose 'Use an existing W&B acco

##### Evaluate

In [ ]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

save_dir = output_dir[:-1] + '/pegasus_test_xsum.txt"'
score_dir = output_dir[:-1] + '/pegasus_test_xsum.json"'

In [ ]:
!python3 $eval_script \
$output_dir \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--task summarization \
--n_obs 20 --bs 2 \
--dump-args

2020-11-19 15:32:06.457858: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
parsed the following generate kwargs: {'length_penalty': 0.8}
100% 1/1 [00:11<00:00, 11.55s/it]
{'rouge1': 34.2593, 'rouge2': 17.8868, 'rougeL': 29.6296, 'rougeLsum': 29.6296, 'n_obs': 2, 'runtime': 11, 'seconds_per_sample': 5.5, 'length_penalty': 0.8}
